In [1]:
import numpy as np
import pandas as pd
import scipy
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv(os.path.join('.', 'colombia_data_sample.csv'))
df = df.astype({'dpto':'category', 
                'mpio':'category', 
                'cabecera':'bool',
                'cod_encuestas':'float64',
                'cant_vivienda':'float64',
                'cant_hog':'float64',
                'cant_per':'float64',
                'sexo':'category', 
                'edad':'float64',
                'parentesco':'category', 
                'lug_nac':'category', 
                'vivia_5anos':'category', 
                'vivia_1ano':'category', 
                'enfermo':'category', 
                'atendido':'bool', 
                'calidad_serv':'float64',
                'discapacidad':'bool', 
                'alfabeta':'bool', 
                'nivel_educ':'category', 
                'trabajo':'category', 
                'est_civil':'category', 
                'hnvh':'float64', 
                'hnvm':'float64', 
                'hnvm':'float64', 
                'hvh':'float64', 
                'hvm':'float64', 
                'hfch':'float64', 
                'hfcm':'float64'})
df['date_uhnv'] = pd.to_datetime(df['date_uhnv'], errors='coerce')
df.head()

### Columnas:
1. **dpto**: Departamento. (string)
2. **mpio**: Municipalidad. (string)
3. **cabecera**: ¿Municipalidad cabecera? (boolean)
4. **cod_encuestas**: Código único de encuesta. (numeric)
5. **cant_vivienda**: Cantidad de viviendas por código de encuesta. (numeric)
6. **cant_hog**: Cantidad de hogares por vivienda. (numeric)
7. **cant_per**: Cantidad de personas por hogar. (numeric)
8. **sexo**: Sexo. (category: Hombre, Mujer)
9. **edad**: Edad. (numeric: rangos de 5 años, edad 1 comprende de 0 a 4 años cumplidos y 21 de 100 en adelante)
10. **parentesco**: Parentesco respecto a jefe de hogar. (category: Jefe_Hogar, Pareja, Hijo, Otro_Pariente, Otro_No_Pariente)
11. **lug_nac**: Nacimiento respecto a municipio actual. (category: Lugar_Actual, Lugar_Otro)
12. **vivia_5anos**: Vivienda respecto a municipio actual hace 5 años. (category: Lugar_Actual, Lugar_Otro)
13. **vivia_1ano**: Vivienda respecto a municipio actual hace 1 año. (category: Lugar_Actual, Lugar_Otro)
14. **enfermo**: Ayuda tomada la última vez que estuvo enfermo. NULL no ha estado enfermo en el último mes. (category: Profesional, No_Profesional, Nada)
15. **atendido**: ¿Fue atendido por la IPS? (boolean)
16. **calidad_serv**: Calificación del servicio de la IPS. (numeric)
17. **discapacidad**: ¿Persona discapacitada? (boolean)
18. **alfabeta**: ¿Es alfabeta? (bool)
19. **nivel_educ**: Nivel de educación máximo alcanzado. (category: Basico, Medio, Avanzado, Ninguno)
20. **trabajo**: Estado laboral. (category: Remunerado, No_Remunerado, Estudio, No_Apto)
21. **est_civil**: Estado civil. (category: Emparejado, Separado, Viudo, Soltero)
22. **hnvh**: Cantidad de hijos nacidos vivos hombres. (numeric)
23. **hnvm**: Cantidad de hijas nacidas vivas mujeres. (numeric)
24. **hvh**: Cantidad de hijos vivos hombres. (numeric)
25. **hvm**: Cantidad de hijas vivas mujeres. (numeric)
26. **hfch**: Cantidad de hijos fuera de Colombia hombres. (numeric)
27. **hfcm**: Cantidad de hijas fuera de Colombia mujeres. (numeric)
28. **date_uhnv**: Fecha de último hijo nacido vivo. (date)

## Idea principal
Se busca tomar una infografía base e irla filtrando hasta el punto que teniendo en cuenta las mismas variables se denotan diferencias de sexo en la cantidad de personas.

## Pre
Se asume que a lo largo de las generaciones Colombia será como una foto

## Escena 1
Municipios se dividen en clusters de manera que se confunda la variable socio-económica.

In [3]:
# Aquí irían los clusters, se dividieron entre cabeceras y no cabeceras en este caso
df['cluster'].unique()

array([4, 3, 1, 0, 2], dtype=int64)

# Orden de clusters, 3 - 4 - 0 - 1 - 2 

## Escena 2:
Nacen 2 parejas de niños (M/F) en municipios escogidos dentro de los clusters. Análisis de edad 1 y 2 (0 a 9) en columna "enfermo", se espera que el valor sea distinto a "Nada". (Edad 1 y 2 no poseen información de trabajo)
¿Se debería filtrar por atención no recibida durante la enfermedad?
Análisis de columna "calidad_serv" buscando si existe diferencia entre sexo

No tiene significancia alguna, no se va a mostrar en la ppt. Se sugiere que en el proximo censo se tenga mas datos para educación 

In [17]:
df_slice

,Unnamed: 0,dpto,mpio,cabecera,cod_encuestas,cant_vivienda,cant_hog,cant_per,sexo,edad,...,trabajo,est_civil,hnvh,hnvm,hvh,hvm,hfch,hfcm,date_uhnv,clusters
19,19,8,1,True,400.0,1.0,1.0,1.0,Mujer,3.0,...,Estudio,Soltero,NaN,NaN,NaN,NaN,NaN,NaN,NaT,3
183,183,54,405,True,5410.0,1.0,1.0,1.0,Hombre,3.0,...,Estudio,Soltero,NaN,NaN,NaN,NaN,NaN,NaN,NaT,3
184,184,54,405,True,5474.0,1.0,1.0,1.0,Hombre,3.0,...,Estudio,Soltero,NaN,NaN,NaN,NaN,NaN,NaN,NaT,3
186,186,54,405,True,5509.0,1.0,1.0,1.0,Mujer,3.0,...,Estudio,Soltero,NaN,NaN,NaN,NaN,NaN,NaN,NaT,3
282,282,8,1,True,8611.0,1.0,1.0,1.0,Hombre,3.0,...,Estudio,Soltero,NaN,NaN,NaN,NaN,NaN,NaN,NaT,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440519,440519,17,1,True,950001911.0,1.0,1.0,1.0,Mujer,3.0,...,NaN,Soltero,NaN,NaN,NaN,NaN,NaN,NaN,NaT,3
440526,440526,25,245,True,950001950.0,1.0,1.0,1.0,Mujer,3.0,...,NaN,Soltero,NaN,NaN,NaN,NaN,NaN,NaN,NaT,3
440534,440534,41,1,True,950002092.0,1.0,1.0,1.0,Mujer,3.0,...,NaN,Soltero,NaN,NaN,NaN,NaN,NaN,NaN,NaT,3
440583,440583,5,615,False,950002499.0,1.0,1.0,1.0,Hombre,3.0,...,NaN,Soltero,NaN,NaN,NaN,NaN,NaN,NaN,NaT,3


In [19]:
df_slice[df_slice['sexo'] == 'Mujer']['enfermo'].value_counts()

Profesional       530
No_Profesional    130
Nada                7
Name: enfermo, dtype: int64

In [23]:
for cluster in df['clusters'].unique():
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([1, 2])) & (df['clusters'] == cluster)].copy()
    enfermo_counts_h = df_slice[df_slice['sexo'] == 'Hombre']['enfermo'].value_counts()
    enfermo_counts_m = df_slice[df_slice['sexo'] == 'Mujer']['enfermo'].value_counts()
    porcentaje_no_atend_h = enfermo_counts_h['Profesional'] / enfermo_counts_h.sum()
    porcentaje_no_atend_m = enfermo_counts_m['Profesional'] / enfermo_counts_m.sum()
    print('La cantidad de niños edad 1 que reciben atención médica es {0:.2f}%'.format(porcentaje_no_atend_h * 100))
    print('La cantidad de niñas edad 1 que reciben atención médica es {0:.2f}%'.format(porcentaje_no_atend_m * 100))
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['sexo'], columns = df_slice['enfermo'] == 'Profesional')
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Atención médica {0:s}es independiente de sexo'.format('no ' if p_val < 0.05 else ''))
    crosstab_counts = pd.crosstab(index = df_slice['sexo'], columns = df_slice['calidad_serv'])
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Calidad de atención {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 4
La cantidad de niños edad 1 que reciben atención médica es 81.43%
La cantidad de niñas edad 1 que reciben atención médica es 79.46%
Atención médica es independiente de sexo
Calidad de atención es independiente de sexo

Para el cluster 3
La cantidad de niños edad 1 que reciben atención médica es 85.31%
La cantidad de niñas edad 1 que reciben atención médica es 85.64%
Atención médica es independiente de sexo
Calidad de atención es independiente de sexo

Para el cluster 1
La cantidad de niños edad 1 que reciben atención médica es 81.15%
La cantidad de niñas edad 1 que reciben atención médica es 76.86%
Atención médica es independiente de sexo
Calidad de atención es independiente de sexo

Para el cluster 0
La cantidad de niños edad 1 que reciben atención médica es 83.42%
La cantidad de niñas edad 1 que reciben atención médica es 80.00%
Atención médica es independiente de sexo
Calidad de atención es independiente de sexo

Para el cluster 2
La cantidad de niños edad 1 que re

## Escena 3: 
Edad 3 (10 a 14) se analiza columna "trabajo", se espera tenga el valor de "Estudio". Se filtra población.

In [33]:
df_slice[df_slice['sexo'] == 'Mujer']['trabajo'].value_counts()

Estudio          4397
No_Remunerado     139
Remunerado         25
No_Apto            16
Name: trabajo, dtype: int64

In [36]:
for cluster in df['clusters'].unique():
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'] == 3) & (df['clusters'] == cluster)].copy()
    estudio_counts_h = df_slice[df_slice['sexo'] == 'Hombre']['trabajo'].value_counts()
    estudio_counts_m = df_slice[df_slice['sexo'] == 'Mujer']['trabajo'].value_counts()
    porcentaje_no_estud_h = 1 - (estudio_counts_h[['Estudio', 'No_Remunerado']].sum() / estudio_counts_h.sum())
    porcentaje_no_estud_m = 1 - (estudio_counts_m[['Estudio', 'No_Remunerado']].sum() / estudio_counts_m.sum())
    print('La cantidad de jovenes hombres edad 3 remunerados es {0:.2f}%'.format(porcentaje_no_estud_h * 100))
    print('La cantidad de jovenes mujeres edad 3 remunerados es {0:.2f}%'.format(porcentaje_no_estud_m * 100))
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['sexo'], columns = df_slice['trabajo'].isin(['Estudio', 'No_Remunerado']))
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Estudio {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 4
La cantidad de jovenes hombres edad 3 remunerados es 1.82%
La cantidad de jovenes mujeres edad 3 remunerados es 0.90%
Estudio no es independiente de sexo

Para el cluster 3
La cantidad de jovenes hombres edad 3 remunerados es 0.97%
La cantidad de jovenes mujeres edad 3 remunerados es 0.67%
Estudio no es independiente de sexo

Para el cluster 1
La cantidad de jovenes hombres edad 3 remunerados es 2.51%
La cantidad de jovenes mujeres edad 3 remunerados es 0.94%
Estudio no es independiente de sexo

Para el cluster 0
La cantidad de jovenes hombres edad 3 remunerados es 2.54%
La cantidad de jovenes mujeres edad 3 remunerados es 0.41%
Estudio no es independiente de sexo

Para el cluster 2
La cantidad de jovenes hombres edad 3 remunerados es 3.17%
La cantidad de jovenes mujeres edad 3 remunerados es 2.15%
Estudio es independiente de sexo



In [16]:
df_slice = df[(df['edad'] == 3) & (df['clusters'] == 3)].copy()
crosstab_counts = pd.crosstab(index = df_slice['sexo'], columns = df_slice['trabajo'] == 'Estudio')
_, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
print(crosstab_counts)
print('p_value = {}'.format(p_val))
estudio_counts_h = df_slice[df_slice['sexo'] == 'Hombre']['trabajo'].value_counts()
estudio_counts_m = df_slice[df_slice['sexo'] == 'Mujer']['trabajo'].value_counts()
porcentaje_no_estud_h = 1 - (estudio_counts_h['Estudio'] / estudio_counts_h.sum())
porcentaje_no_estud_m = 1 - (estudio_counts_m['Estudio'] / estudio_counts_m.sum())
print('% hombres = {}'.format(porcentaje_no_estud_h))
print('% mujeres = {}'.format(porcentaje_no_estud_m))

trabajo  False  True 
sexo                 
Hombre     737   7963
Mujer      590   7741
p_value = 0.0008003888012085088
% hombres = 0.02114320835894279
% mujeres = 0.021489065857666523


## Escena 4:
Edad 3 (10 a 14) se espera que no haya relación entre los hijos que tienen y su actividad académica, se tiene en cuenta cuadro de contingencia entre hijos nacidos ("hnvh" + "hnvm") respecto a "trabajo", si es significativo se filtra población.

In [6]:
for cluster in df['clusters'].unique():
    print('Para el cluster {0:d}'.format(cluster))
    # Slicing
    df_slice = df[(df['edad'] == 3) & (df['clusters'] == cluster)].copy()
    df_slice['hnvt'] = df_slice['hnvh'].fillna(0.0) + df_slice['hnvm'].fillna(0.0)
    max_digit = len(str(max(df_slice.groupby('hnvt')['hnvt'].count())))
    for ind, grouped in df_slice.groupby('hnvt'):
        # EDA
        estudio_counts = grouped['trabajo'].value_counts()
        porcentaje_no_estud = 1 - (estudio_counts['Estudio'] / estudio_counts.sum())
        print('En edad 3, {0:<{3:.0f}.0f} jovenes tienen {1:.0f} hijo(s), de los cuales el {2:>6.2f}% abandonan sus estudios' \
              .format(grouped['hnvt'].count(), ind, porcentaje_no_estud * 100, max_digit))
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['hnvt'], columns = df_slice['trabajo'] == 'Estudio')
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Estudio {0:s}es independiente de cantidad de hijos\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 4
En edad 3, 9922 jovenes tienen 0 hijo(s), de los cuales el   3.72% abandonan sus estudios
En edad 3, 8    jovenes tienen 1 hijo(s), de los cuales el  62.50% abandonan sus estudios
En edad 3, 1    jovenes tienen 2 hijo(s), de los cuales el 100.00% abandonan sus estudios
En edad 3, 2    jovenes tienen 3 hijo(s), de los cuales el 100.00% abandonan sus estudios
Estudio no es independiente de cantidad de hijos

Para el cluster 3
En edad 3, 17017 jovenes tienen 0 hijo(s), de los cuales el   2.07% abandonan sus estudios
En edad 3, 11    jovenes tienen 1 hijo(s), de los cuales el  63.64% abandonan sus estudios
En edad 3, 1     jovenes tienen 2 hijo(s), de los cuales el 100.00% abandonan sus estudios
En edad 3, 2     jovenes tienen 3 hijo(s), de los cuales el 100.00% abandonan sus estudios
Estudio no es independiente de cantidad de hijos

Para el cluster 1
En edad 3, 6177 jovenes tienen 0 hijo(s), de los cuales el   5.20% abandonan sus estudios
En edad 3, 9    jovenes tienen 1

In [38]:
for cluster in df['clusters'].unique():
    print('Para el cluster {0:d}'.format(cluster))
    # Slicing
    df_slice = df[(df['edad'] == 3) & (df['clusters'] == cluster)].copy()
    df_slice['hnvt'] = df_slice['hnvh'].fillna(0.0) + df_slice['hnvm'].fillna(0.0)
    df_slice['hnvt'] = df_slice['hnvt'] > 0
    estudio_counts = df_slice[df_slice['hnvt']]['trabajo'].value_counts()
    porcentaje_no_estud = 1 - (estudio_counts['Estudio'] / estudio_counts.sum())
    print(porcentaje_no_estud * 100)
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['hnvt'], columns = df_slice['trabajo'] == 'Estudio')
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Estudio {0:s}es independiente de cantidad de hijos\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 4
72.72727272727273
Estudio no es independiente de cantidad de hijos

Para el cluster 3
71.42857142857143
Estudio no es independiente de cantidad de hijos

Para el cluster 1
93.33333333333333
Estudio no es independiente de cantidad de hijos

Para el cluster 0
33.333333333333336
Estudio es independiente de cantidad de hijos

Para el cluster 2
66.66666666666667
Estudio es independiente de cantidad de hijos



## Escena 5:
Edad 4 (15 a 19) los jovenes que ya mantienen su hogar deberían ser alfabetas para poder trabajar y mantenerse dignamente, se tiene encuenta "parentesco", se espera que si tiene el valor "Jefe_Hogar" o "Pareja" entonces "alfabeta" sea "True". ¿Se filtra población?

In [30]:
for cluster in df['clusters'].unique():
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'] == 4) & (df['clusters'] == cluster) & ~df['alfabeta']].copy()
    jefe_counts_h = df_slice[(df_slice['sexo'] == 'Hombre')]['parentesco'].value_counts()
    jefe_counts_m = df_slice[(df_slice['sexo'] == 'Mujer')]['parentesco'].value_counts()
    porcentaje_jefe_h = jefe_counts_h[['Jefe_Hogar']].sum() / jefe_counts_h.sum()
    porcentaje_jefe_m = jefe_counts_m[['Jefe_Hogar']].sum() / jefe_counts_m.sum()
    print('La cantidad de jovenes hombres analfabetas de edad 4 que deben mantener un hogar es {0:.2f}%'.format(porcentaje_jefe_h * 100))
    print('La cantidad de jovenes mujeres analfabetas de edad 4 que deben mantener un hogar es {0:.2f}%'.format(porcentaje_jefe_m * 100))
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['parentesco'].isin(['Jefe_Hogar']), columns = df_slice['sexo'])
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Manutención de hogar {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 4
La cantidad de jovenes hombres analfabetas de edad 4 que deben mantener un hogar es 15.13%
La cantidad de jovenes mujeres analfabetas de edad 4 que deben mantener un hogar es 13.70%
Manutención de hogar es independiente de sexo

Para el cluster 3
La cantidad de jovenes hombres analfabetas de edad 4 que deben mantener un hogar es 20.21%
La cantidad de jovenes mujeres analfabetas de edad 4 que deben mantener un hogar es 19.32%
Manutención de hogar es independiente de sexo

Para el cluster 1
La cantidad de jovenes hombres analfabetas de edad 4 que deben mantener un hogar es 12.79%
La cantidad de jovenes mujeres analfabetas de edad 4 que deben mantener un hogar es 5.36%
Manutención de hogar es independiente de sexo

Para el cluster 0
La cantidad de jovenes hombres analfabetas de edad 4 que deben mantener un hogar es 11.76%
La cantidad de jovenes mujeres analfabetas de edad 4 que deben mantener un hogar es 12.50%
Manutención de hogar es independiente de sexo

Para el clust

## Escena 6
Edad 5 (20 a 25) luego de haber logrado el camino de la educación, de los no hijos y de la salud, ahora se evalúan las condiciones de vida. Empezamos por analizar cuántos educados no poseen remuneración.

In [8]:
for cluster in df['clusters'].unique():
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([5, 6, 7])) & (df['clusters'] == cluster) & (df['trabajo'].isin(['Remunerado', 'No_Remunerado']))].copy()
    trabajo_counts_h = df_slice[(df_slice['sexo'] == 'Hombre')]['trabajo'].value_counts()
    trabajo_counts_m = df_slice[(df_slice['sexo'] == 'Mujer')]['trabajo'].value_counts()
    porcentaje_trabajo_h = trabajo_counts_h['Remunerado'].sum() / trabajo_counts_h.sum()
    porcentaje_trabajo_m = trabajo_counts_m['Remunerado'].sum() / trabajo_counts_m.sum()
    print('La cantidad de jovenes hombres educados de edad 5 que tienen trabajo remunerado es {0:.2f}%'.format(porcentaje_trabajo_h * 100))
    print('La cantidad de jovenes mujeres educadas de edad 5 que tienen trabajo remunerado es {0:.2f}%'.format(porcentaje_trabajo_m * 100))
    # Estadística
    crosstab_counts = pd.crosstab(index = df_slice['trabajo'], columns = df_slice['sexo'])
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Remuneración {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 4
La cantidad de jovenes hombres educados de edad 5 que tienen trabajo remunerado es 83.23%
La cantidad de jovenes mujeres educadas de edad 5 que tienen trabajo remunerado es 42.61%
Remuneración no es independiente de sexo

Para el cluster 3
La cantidad de jovenes hombres educados de edad 5 que tienen trabajo remunerado es 86.73%
La cantidad de jovenes mujeres educadas de edad 5 que tienen trabajo remunerado es 66.96%
Remuneración no es independiente de sexo

Para el cluster 1
La cantidad de jovenes hombres educados de edad 5 que tienen trabajo remunerado es 79.24%
La cantidad de jovenes mujeres educadas de edad 5 que tienen trabajo remunerado es 27.28%
Remuneración no es independiente de sexo

Para el cluster 0
La cantidad de jovenes hombres educados de edad 5 que tienen trabajo remunerado es 74.54%
La cantidad de jovenes mujeres educadas de edad 5 que tienen trabajo remunerado es 32.17%
Remuneración no es independiente de sexo

Para el cluster 2
La cantidad de jovenes

Edades 25 - 60, ver por hombre y mujer inmigracion.

Edades 25 - 60, mujeres casadas tienen menos remunerado que hombres casados.

Edad 50+ tipo de educacion.